In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = False
from pycaret.classification import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#QUERY QUE DA LA PROPRCION DE USUARIOS CALIFICADOS POR EL MODELO DE CALIDAD SEGUN CHURN DEL ENDING
query=f'''
SELECT A.PERIODO_PROCESO_CODIGO,
       B.PREDICCION,
       COUNT(*) AS TOTAL
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
JOIN AGG_DL_PJ_QUALITY B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO >=202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE='CHURN PORTACION'
      AND TO_NUMBER(TO_CHAR(A.FECHA_ALTA,'YYYYMMDD'))=B.FECHA_ALTA
GROUP BY A.PERIODO_PROCESO_CODIGO,B.PREDICCION
ORDER BY A.PERIODO_PROCESO_CODIGO,B.PREDICCION
'''



c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [2]:
#query que genera la base de 
query=f'''
SELECT A.PERIODO_PROCESO_CODIGO,
       A.SUBSCRIBER_ID,
       A.MOVIMIENTO_TIPO_NOMBRE,
       CASE
            WHEN A.MOVIMIENTO_TIPO_NOMBRE='PERMANECE' THEN 0 
            ELSE 1
       END AS PORTACION,
       A.OPERADOR_PORTA_ORIGEN,
       CASE
            WHEN A.OPERADOR_PORTA_ORIGEN IS NULL THEN 1 
            ELSE 0 
       END AS LINEA_NUEVA,
       A.FECHA_ALTA,
       ROUND(TO_DATE(TO_CHAR(20221130), 'YYYY/MM/DD')-A.FECHA_ALTA,0) AS  ANTIGUEDAD,
       COALESCE(B.PREDICCION,0) AS MALA_CALIDAD_GROSS,
       COALESCE(B.PROBABILIDAD,0) AS PROBA_CALIDAD_GROSS,
       B.CONSULTOR_NOMBRE_NIVEL_8,
       ROUND(C.CHURN_ACUM*100/C.GROSS_ACUM,0) AS CHURN_RATIO_DEALER
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
LEFT JOIN (SELECT * FROM AGG_DL_PJ_QUALITY WHERE PROBABILIDAD<=100 )B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN (SELECT * FROM DWH_CONSULTA_WOM.AA_SEGUIMIENTO_DEALERS  WHERE SERVICIO='Prepaid'
      AND PERIODO_PROCESO_CODIGO=202211-1 )C ON B.CONSULTOR_NOMBRE_NIVEL_8=C.DEALER
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      AND A.ESTADO IN ('Active','Termination') 
      
'''
base=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
len(base)

1171334

In [4]:
#variables de la base portacion, linea nueva, antiguedad,mala calidad gross,proba calidad gross y churn ratio del dealer
base.to_csv('base.csv')

In [5]:
base

,PERIODO_PROCESO_CODIGO,SUBSCRIBER_ID,MOVIMIENTO_TIPO_NOMBRE,PORTACION,OPERADOR_PORTA_ORIGEN,LINEA_NUEVA,FECHA_ALTA,ANTIGUEDAD,MALA_CALIDAD_GROSS,PROBA_CALIDAD_GROSS,CONSULTOR_NOMBRE_NIVEL_8,CHURN_RATIO_DEALER
0,202211,1217173137,PERMANECE,0,None,1,2021-12-28 21:02:42,336.0,0,0,None,NaN
1,202211,1217175364,PERMANECE,0,None,1,2021-12-27 10:12:47,338.0,0,0,None,NaN
2,202211,1219682999,PERMANECE,0,None,1,2021-12-10 11:45:18,355.0,0,0,None,NaN
3,202211,1174616021,PERMANECE,0,None,1,2021-08-11 12:16:28,475.0,0,0,None,NaN
4,202211,1141318992,PERMANECE,0,None,1,2021-08-10 16:37:04,476.0,0,0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1171329,202211,2212556772,CHURN PORTACION,1,AVANTEL,0,2022-02-21 14:58:42,281.0,0,13,Dina Fernanda Serrano Cristian,NaN
1171330,202211,2209554923,CHURN PORTACION,1,TELEFONICA,0,2022-06-10 08:52:43,173.0,0,22,Emergia Customer Care,NaN
1171331,202211,2261090200,CHURN PORTACION,1,None,1,2022-07-08 21:09:31,144.0,0,8,Sin Informacion,35.0
1171332,202211,1311749633,CHURN PORTACION,1,None,1,2022-09-28 00:00:00,63.0,1,83,Tobarcell Popayan Sas,61.0


In [6]:
#varaile si viene de pospago 
query=f'''
SELECT * FROM (
SELECT A.SUBSCRIBER_ID,
       A.MOVIMIENTO_TIPO_NOMBRE,
       CASE
            WHEN A.MOVIMIENTO_TIPO_NOMBRE='GROSS ADDS NUEVA' THEN 0 
            ELSE 1 
       END AS VIENE_POSPAGO,
       ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY PERIODO_PROCESO_CODIGO DESC) AS RN 
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.SERVICIO='Prepaid'
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('GROSS ADDS MIGRACION','GROSS ADDS NUEVA')   
      AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
      )
      WHERE RN=1
'''
viene_pospago=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
len(viene_pospago)

1166680

In [8]:
viene_pospago.to_csv('viene_pospago.csv')

In [9]:
viene_pospago

,SUBSCRIBER_ID,MOVIMIENTO_TIPO_NOMBRE,VIENE_POSPAGO,RN
0,192553045,GROSS ADDS NUEVA,0,1
1,192553580,GROSS ADDS NUEVA,0,1
2,192554318,GROSS ADDS NUEVA,0,1
3,192554585,GROSS ADDS NUEVA,0,1
4,192554955,GROSS ADDS NUEVA,0,1
...,...,...,...,...
1166675,2317646705,GROSS ADDS MIGRACION,1,1
1166676,2317694056,GROSS ADDS MIGRACION,1,1
1166677,2317724567,GROSS ADDS MIGRACION,1,1
1166678,2317981288,GROSS ADDS NUEVA,0,1


In [10]:
#quejas historicas 
query=f'''
SELECT A.SUBSCRIBER_ID,COUNT(*) AS TOTAL_QUEJAS 
FROM CR_DL_PQR A
WHERE A.SERVICIO='Prepaid'
      AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
      AND A.SERVICE_TYPE1='QUEJA'
      AND A.PERIODO_PROCESO_CODIGO<=202211
GROUP BY A.SUBSCRIBER_ID
'''
quejas_historicas=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
len(quejas_historicas)

43210

In [12]:
quejas_historicas.to_csv('quejas_historicas.csv')

In [13]:
quejas_historicas

,SUBSCRIBER_ID,TOTAL_QUEJAS
0,1306851696,1
1,1208232457,2
2,1115546968,2
3,1136667341,1
4,1221000510,1
...,...,...
43205,2238369824,1
43206,1190582904,3
43207,1262919415,7
43208,2190774677,1


In [18]:
#llamadas de la competencia
query=f'''
SELECT  K.SUBSCRIBER_ID,
        SUM(K.DURACION_SEG) AS TOTAL_SEGUNDOS_COMPETENCIA,
        COUNT(*) AS TOTAL_LLAMADAS_COMPETENCIA,
        ROUND(SUM(K.DURACION_SEG)/COUNT(*),0) AS PROMEDIO_LLAMADA_COMPETENCIA
FROM (SELECT  SUBSCRIBER_ID,
        PERIODO_PROCESO_CODIGO,
        NUMERO_ORIGEN,
        DURACION_SEG
FROM  CDR_BODEGA_WOM.AGR_TRAFICO_MOU_COMPETENCIA
WHERE PERIODO_PROCESO_CODIGO BETWEEN 20221101 AND 20221130
AND SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
) K
GROUP BY K.SUBSCRIBER_ID
'''
llamadas_competencia=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
len(llamadas_competencia)

242608

In [20]:
llamadas_competencia.to_csv('llamadas_competencia.csv')

In [21]:
llamadas_competencia

,SUBSCRIBER_ID,TOTAL_SEGUNDOS_COMPETENCIA,TOTAL_LLAMADAS_COMPETENCIA,PROMEDIO_LLAMADA_COMPETENCIA
0,1319044345,159,5,32
1,1264916534,8,2,4
2,1126418471,919,29,32
3,1230975168,82,9,9
4,1145601046,37,3,12
...,...,...,...,...
242603,1131764529,2,1,2
242604,1157170947,82,4,21
242605,1190861375,80,2,40
242606,1129786276,41,2,21


In [22]:
#generamos la hipotesis de dia de la semana pivoteando los dias y dejando como valor las cantidades
query=f'''
SELECT SUBSCRIBER_ID,DIA_SEMANA_EF,TOTAL_DIA FROM (
WITH BASE AS (SELECT  A.SUBSCRIBER_ID,
        to_number(to_char(B.FECHA_RECARGA,'D','NLS_DATE_LANGUAGE=SPANISH')) AS DIA_SEMANA_EF
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO in (202211) 
        AND A.SERVICIO='Prepaid'
        AND B.SERVICIO='Prepaid'
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
)
SELECT  SUBSCRIBER_ID,
        DIA_SEMANA_EF,
        COUNT(DIA_SEMANA_EF) AS TOTAL_DIA,
        ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY COUNT(DIA_SEMANA_EF) DESC) AS RN
FROM BASE
GROUP BY SUBSCRIBER_ID,DIA_SEMANA_EF ) K

'''
recargas_paquetes_dia_semana=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
#pivotear la tabla y poner 0 los nulos
recargas_paquetes_dia_semana=recargas_paquetes_dia_semana.pivot_table(index='SUBSCRIBER_ID',columns='DIA_SEMANA_EF',values='TOTAL_DIA')
recargas_paquetes_dia_semana=recargas_paquetes_dia_semana.fillna(0)

In [24]:
#resetear index y seleccionar las columnas 
recargas_paquetes_dia_semana.reset_index()
recargas_paquetes_dia_semana=recargas_paquetes_dia_semana.reset_index()
recargas_paquetes_dia_semana.columns=['SUBSCRIBER_ID','1','2','3','4','5','6','7']
recargas_paquetes_dia_semana=recargas_paquetes_dia_semana[['SUBSCRIBER_ID','1','2','3','4','5','6','7']]


In [25]:
recargas_paquetes_dia_semana['RECARGAS_FINDE']=recargas_paquetes_dia_semana['1']+recargas_paquetes_dia_semana['7']+recargas_paquetes_dia_semana['6']
recargas_paquetes_dia_semana['RECARGAS_ENTRE_SEMANA']=recargas_paquetes_dia_semana['2']+recargas_paquetes_dia_semana['3']+recargas_paquetes_dia_semana['4']+recargas_paquetes_dia_semana['5']

In [26]:
len(recargas_paquetes_dia_semana)

1217522

In [27]:
recargas_paquetes_dia_semana.to_csv('recargas_paquetes_dia_semana.csv')

In [28]:
recargas_paquetes_dia_semana

,SUBSCRIBER_ID,1,2,3,4,5,6,7,RECARGAS_FINDE,RECARGAS_ENTRE_SEMANA
0,192553580,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,192554318,0.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0,5.0
2,192554585,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,192554955,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,192555940,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...
1217517,2316649761,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1217518,2317361904,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1217519,2317646705,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1217520,2317724567,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
#generamos la hipotesis de franja horarias
query=f'''
SELECT SUBSCRIBER_ID,FRANJA_RECARGAS,TOTAL_RECARGAS_FRANJA FROM(
WITH BASE AS(
SELECT  A.SUBSCRIBER_ID,
        TO_CHAR(FECHA_RECARGA, 'HH24') as FRANJA_RECARGAS
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND B.SERVICIO='Prepaid'
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
        )

SELECT  SUBSCRIBER_ID,
        FRANJA_RECARGAS,
        COUNT(FRANJA_RECARGAS) AS TOTAL_RECARGAS_FRANJA,
        ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY COUNT(FRANJA_RECARGAS) DESC) AS RN
FROM BASE
GROUP BY SUBSCRIBER_ID,FRANJA_RECARGAS) K 
'''
recargas_franja_horaria=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [30]:
#pivotear la tabla y poner 0 los nulos
recargas_franja_horaria=recargas_franja_horaria.pivot_table(index='SUBSCRIBER_ID',columns='FRANJA_RECARGAS',values='TOTAL_RECARGAS_FRANJA')
recargas_franja_horaria=recargas_franja_horaria.fillna(0)

In [31]:
#resetear index
recargas_franja_horaria.reset_index()
recargas_franja_horaria=recargas_franja_horaria.reset_index()
recargas_franja_horaria.columns=['SUBSCRIBER_ID','00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
recargas_franja_horaria

,SUBSCRIBER_ID,00,01,02,03,04,05,06,07,08,...,14,15,16,17,18,19,20,21,22,23
0,192553580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,192554318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,192554585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,192554955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,192555940,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217517,2316649761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1217518,2317361904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1217519,2317646705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1217520,2317724567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#CREACION HIPOTESIS  
recargas_franja_horaria['RECARGA_HORA_9_17']=recargas_franja_horaria['09']+recargas_franja_horaria['10']+recargas_franja_horaria['11']+recargas_franja_horaria['12']+recargas_franja_horaria['13']+recargas_franja_horaria['14']+recargas_franja_horaria['15']+recargas_franja_horaria['16']+recargas_franja_horaria['17']
recargas_franja_horaria['RECARGA_HORA_18_8']=recargas_franja_horaria['18']+recargas_franja_horaria['19']+recargas_franja_horaria['20']+recargas_franja_horaria['21']+recargas_franja_horaria['22']+recargas_franja_horaria['23']+recargas_franja_horaria['00']+recargas_franja_horaria['01']+recargas_franja_horaria['02']+recargas_franja_horaria['03']+recargas_franja_horaria['04']+recargas_franja_horaria['05']+recargas_franja_horaria['06']+recargas_franja_horaria['07']+recargas_franja_horaria['08']


In [33]:
#SELECCION SOLO DE LA HIPOTESIS

recargas_franja_horaria=recargas_franja_horaria[['SUBSCRIBER_ID','RECARGA_HORA_9_17','RECARGA_HORA_18_8']]


In [34]:
recargas_franja_horaria.to_csv('recargas_franja_horaria_final.csv')

In [35]:
recargas_franja_horaria

,SUBSCRIBER_ID,RECARGA_HORA_9_17,RECARGA_HORA_18_8
0,192553580,1.0,0.0
1,192554318,5.0,1.0
2,192554585,1.0,0.0
3,192554955,1.0,1.0
4,192555940,2.0,1.0
...,...,...,...
1217517,2316649761,2.0,0.0
1217518,2317361904,1.0,1.0
1217519,2317646705,1.0,0.0
1217520,2317724567,0.0,1.0


In [37]:
#query para variables de recargas y paquetes historicas
query=f'''
SELECT  A.SUBSCRIBER_ID,
        B.CANTIDAD_RECARGAS,
        B.VALOR_RECARGAS,
        B.PROMEDIO_RECARGA,
        C.VALOR_PRIMERA_RECARGA,
        G.COMPRO_PAQUETE,
        G.CANTIDAD_PAQUETES,
        G.VALOR_PAQUETES,
        G.PROMEDIO_PAQUETES
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN (WITH BASE AS (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.TIEMPO_RECARGA_DK,
        B.VALOR_CARGA
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND B.SERVICIO='Prepaid'
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      ) 
)
SELECT  SUBSCRIBER_ID,
        COUNT(SUBSCRIBER_ID) AS CANTIDAD_RECARGAS,
        SUM(VALOR_CARGA) AS VALOR_RECARGAS,
        SUM(VALOR_CARGA)/COUNT(SUBSCRIBER_ID) AS PROMEDIO_RECARGA
FROM BASE
GROUP BY SUBSCRIBER_ID) B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN (SELECT
    SUBSCRIBER_ID,
    VALOR_CARGA AS VALOR_PRIMERA_RECARGA
FROM (
SELECT
    A.SUBSCRIBER_ID,
    A.MOVIMIENTO_TIPO_NOMBRE,
    TO_NUMBER(TO_CHAR(A.FECHA_ALTA,'YYYYMMDD'))AS FECHA_ALTA,
    B.TIEMPO_RECARGA_DK,
    B.VALOR_CARGA,
    ROW_NUMBER() OVER(PARTITION BY A.SUBSCRIBER_ID ORDER BY B.TIEMPO_RECARGA_DK ASC) AS RN
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202211
    AND A.SERVICIO='Prepaid'
    AND B.SERVICIO='Prepaid'
    AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
       )
)
WHERE RN=1) C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
LEFT JOIN (WITH BASE AS (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        C.TIEMPO_PAQUETE_DK,
        C.VALOR
FROM DWH_BODEGA_WOM.FCT_PAQUETES C
RIGHT JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON C.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211 
        AND A.SERVICIO='Prepaid'
        AND  A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
        AND C.SERVICIO='Prepaid'
)
SELECT  SUBSCRIBER_ID,
        CASE
            WHEN SUM(VALOR)>0 THEN 1
            ELSE 0
        END COMPRO_PAQUETE,
        CASE
            WHEN SUM(VALOR)=0 THEN 0
            ELSE COUNT(VALOR)
        END CANTIDAD_PAQUETES,
        COALESCE(SUM(VALOR),0) AS VALOR_PAQUETES,
        SUM(VALOR)/COUNT(SUBSCRIBER_ID) AS PROMEDIO_PAQUETES
        
FROM BASE
GROUP BY SUBSCRIBER_ID ) G ON A.SUBSCRIBER_ID=G.SUBSCRIBER_ID
--------------------------
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid' 
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
'''
recargas_paquetes_hist=cst.consulta(query)

In [38]:
recargas_paquetes_hist=recargas_paquetes_hist.fillna(0)

In [ ]:
#query para variables de recargas y paquetes ultimo mes
query=f'''
SELECT  A.SUBSCRIBER_ID,
        B.CANTIDAD_RECARGAS,
        B.VALOR_RECARGAS,
        B.PROMEDIO_RECARGA,
        C.VALOR_PRIMERA_RECARGA,
        G.COMPRO_PAQUETE,
        G.CANTIDAD_PAQUETES,
        G.VALOR_PAQUETES,
        G.PROMEDIO_PAQUETES
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN (WITH BASE AS (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.TIEMPO_RECARGA_DK,
        B.VALOR_CARGA
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND B.SERVICIO='Prepaid'
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      ) 
)
SELECT  SUBSCRIBER_ID,
        COUNT(SUBSCRIBER_ID) AS CANTIDAD_RECARGAS,
        SUM(VALOR_CARGA) AS VALOR_RECARGAS,
        SUM(VALOR_CARGA)/COUNT(SUBSCRIBER_ID) AS PROMEDIO_RECARGA
FROM BASE
GROUP BY SUBSCRIBER_ID) B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN (SELECT
    SUBSCRIBER_ID,
    VALOR_CARGA AS VALOR_PRIMERA_RECARGA
FROM (
SELECT
    A.SUBSCRIBER_ID,
    A.MOVIMIENTO_TIPO_NOMBRE,
    TO_NUMBER(TO_CHAR(A.FECHA_ALTA,'YYYYMMDD'))AS FECHA_ALTA,
    B.TIEMPO_RECARGA_DK,
    B.VALOR_CARGA,
    ROW_NUMBER() OVER(PARTITION BY A.SUBSCRIBER_ID ORDER BY B.TIEMPO_RECARGA_DK ASC) AS RN
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202211
    AND A.SERVICIO='Prepaid'
    AND B.SERVICIO='Prepaid'
    AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
       )
)
WHERE RN=1) C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
LEFT JOIN (WITH BASE AS (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        C.TIEMPO_PAQUETE_DK,
        C.VALOR
FROM DWH_BODEGA_WOM.FCT_PAQUETES C
RIGHT JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON C.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211 
        AND A.SERVICIO='Prepaid'
        AND  A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
        AND C.SERVICIO='Prepaid'
)
SELECT  SUBSCRIBER_ID,
        CASE
            WHEN SUM(VALOR)>0 THEN 1
            ELSE 0
        END COMPRO_PAQUETE,
        CASE
            WHEN SUM(VALOR)=0 THEN 0
            ELSE COUNT(VALOR)
        END CANTIDAD_PAQUETES,
        COALESCE(SUM(VALOR),0) AS VALOR_PAQUETES,
        SUM(VALOR)/COUNT(SUBSCRIBER_ID) AS PROMEDIO_PAQUETES
        
FROM BASE
GROUP BY SUBSCRIBER_ID ) G ON A.SUBSCRIBER_ID=G.SUBSCRIBER_ID
--------------------------
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid' 
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
'''
recargas_paquetes_1m=cst.consulta(query)

In [ ]:
recargas_paquetes_1m=recargas_paquetes_1m.fillna(0)

In [ ]:
#query para variables de recargas y paquetes historicas
query=f'''
SELECT  A.SUBSCRIBER_ID,
        B.CANTIDAD_RECARGAS,
        B.VALOR_RECARGAS,
        B.PROMEDIO_RECARGA,
        C.VALOR_PRIMERA_RECARGA,
        G.COMPRO_PAQUETE,
        G.CANTIDAD_PAQUETES,
        G.VALOR_PAQUETES,
        G.PROMEDIO_PAQUETES
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN (WITH BASE AS (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.TIEMPO_RECARGA_DK,
        B.VALOR_CARGA
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND B.SERVICIO='Prepaid'
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      ) 
)
SELECT  SUBSCRIBER_ID,
        COUNT(SUBSCRIBER_ID) AS CANTIDAD_RECARGAS,
        SUM(VALOR_CARGA) AS VALOR_RECARGAS,
        SUM(VALOR_CARGA)/COUNT(SUBSCRIBER_ID) AS PROMEDIO_RECARGA
FROM BASE
GROUP BY SUBSCRIBER_ID) B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN (SELECT
    SUBSCRIBER_ID,
    VALOR_CARGA AS VALOR_PRIMERA_RECARGA
FROM (
SELECT
    A.SUBSCRIBER_ID,
    A.MOVIMIENTO_TIPO_NOMBRE,
    TO_NUMBER(TO_CHAR(A.FECHA_ALTA,'YYYYMMDD'))AS FECHA_ALTA,
    B.TIEMPO_RECARGA_DK,
    B.VALOR_CARGA,
    ROW_NUMBER() OVER(PARTITION BY A.SUBSCRIBER_ID ORDER BY B.TIEMPO_RECARGA_DK ASC) AS RN
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A ON B.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202211
    AND A.SERVICIO='Prepaid'
    AND B.SERVICIO='Prepaid'
    AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
       )
)
WHERE RN=1) C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
LEFT JOIN (WITH BASE AS (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        C.TIEMPO_PAQUETE_DK,
        C.VALOR
FROM DWH_BODEGA_WOM.FCT_PAQUETES C
RIGHT JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
ON C.SUBSCRIBER_ID=A.SUBSCRIBER_ID
WHERE   A.PERIODO_PROCESO_CODIGO=202211 
        AND A.SERVICIO='Prepaid'
        AND  A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
        AND C.SERVICIO='Prepaid'
)
SELECT  SUBSCRIBER_ID,
        CASE
            WHEN SUM(VALOR)>0 THEN 1
            ELSE 0
        END COMPRO_PAQUETE,
        CASE
            WHEN SUM(VALOR)=0 THEN 0
            ELSE COUNT(VALOR)
        END CANTIDAD_PAQUETES,
        COALESCE(SUM(VALOR),0) AS VALOR_PAQUETES,
        SUM(VALOR)/COUNT(SUBSCRIBER_ID) AS PROMEDIO_PAQUETES
        
FROM BASE
GROUP BY SUBSCRIBER_ID ) G ON A.SUBSCRIBER_ID=G.SUBSCRIBER_ID
--------------------------
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid' 
        AND A.SUBSCRIBER_ID IN (
      SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
'''
recargas_paquetes_3m=cst.consulta(query)

In [ ]:
recargas_paquetes_3m=recargas_paquetes_3m.fillna(0)

In [ ]:
#TRAFICO DE DATOS DEL ULTIMO MES , dias que navego y total megas consumidas
query=f'''
SELECT SUBSCRIBER_ID,
       ROUND( SUM(BYTES_TOTALES)/1024/1024,0) AS MB_TOTALES,
       COUNT(DISTINCT(PERIODO_PROCESO_CODIGO)) AS DIAS_NAV
FROM AGG_DL_DATA B
WHERE B.SUBSCRIBER_ID IN (SELECT  A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')))
AND B.SERVICIO='Prepaid'
AND B.PERIODO_PROCESO_CODIGO BETWEEN 20221101 AND 20221130 
GROUP BY SUBSCRIBER_ID
'''

trafico_datos=cst.consulta(query) 

In [ ]:
query=f'''
SELECT  SUBSCRIBER_ID,
        SUM(LLAMADA_NORMAL)AS LLAMADAS_NORMAL_ENTRANTE,
        COUNT(DISTINCT(PERIODO_PROCESO_CODIGO)) AS DIAS_ENTRANTE
FROM AGG_DL_VOICE B 
WHERE B.SUBSCRIBER_ID IN (SELECT  A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')))
AND B.SERVICIO='Prepaid'
AND B.PERIODO_PROCESO_CODIGO BETWEEN 20221020 AND 20221130 
AND B.SENTIDO='ENTRANTE'
GROUP BY SUBSCRIBER_ID
'''
entrantes=cst.consulta(query)   


In [ ]:
#  el trafico de voz de salientes si importa la duracion  y las llamadas error que pudo tener
query=f'''
SELECT SUBSCRIBER_ID,
    
        SUM(DURACION_CALLS) AS DURACION_LLAMADAS_SALIENTE,
        SUM(LLAMADA_NORMAL)AS LLAMADAS_NORMAL_SALIENTE,
        SUM(LLAMADA_ERROR) AS LLAMADA_ERROR_SALIENTE,
        COUNT(DISTINCT(PERIODO_PROCESO_CODIGO)) AS DIAS_SALIENTE
FROM AGG_DL_VOICE B 
WHERE B.SUBSCRIBER_ID IN (SELECT  A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')))
AND B.SERVICIO='Prepaid'
AND B.PERIODO_PROCESO_CODIGO BETWEEN 20221020 AND 20221130 
AND B.SENTIDO='SALIENTE'
GROUP BY SUBSCRIBER_ID
'''
salientes=cst.consulta(query)

In [ ]:
#mensajes
query=f'''
WITH BASE AS (
SELECT A.NUM_DESTINO,
       COUNT(*) AS MENSAJES_COMPETENCIA
FROM DP_TRAFICO_SMS A
WHERE A.NUM_DESTINO  IN (
      SELECT A.MSISDN
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE')
      )
AND A.FECHA_ENVIO BETWEEN 20221001 AND 20221031
AND A.TOPICOS='OFERTA'
GROUP BY NUM_DESTINO ) 
,
BASE2 AS (SELECT SUBSCRIBER_ID,MSISDN
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO =202211
      AND A.SERVICIO='Prepaid'  
      AND A.MOVIMIENTO_TIPO_NOMBRE IN ('CHURN PORTACION','PERMANECE'))
SELECT B.*,A.MENSAJES_COMPETENCIA
FROM BASE A JOIN BASE2 B ON A.NUM_DESTINO=B.MSISDN
'''
sms_competencia=cst.consulta(query)

In [2]:
#lectura de datos 
base = pd.read_csv('base.csv', sep=',')
viene_pospago = pd.read_csv('viene_pospago.csv', sep=',')
quejas_historicas = pd.read_csv('quejas_historicas.csv', sep=',')
llamadas_competencia = pd.read_csv('llamadas_competencia.csv', sep=',')
recargas_paquetes_dia_semana = pd.read_csv('recargas_paquetes_dia_semana.csv', sep=',')
recargas_franja_horaria = pd.read_csv('recargas_franja_horaria_final.csv', sep=',')
recargas_paquetes_hist= pd.read_csv('recargas_paquetes_hist.csv', sep=',')
trafico_datos = pd.read_csv('trafico_datos.csv', sep=',')
sms_competencia = pd.read_csv('sms_competencia.csv', sep=',')
#entrantes y salientes no cargaron 

In [3]:
#junte de bases
df_insumos=base.merge(viene_pospago,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(quejas_historicas,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(llamadas_competencia,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(recargas_paquetes_dia_semana,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(recargas_franja_horaria,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(recargas_paquetes_hist,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(trafico_datos,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(sms_competencia,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.fillna(0)

In [ ]:
df_insumos.columns

In [4]:
df_resultado=df_insumos[['PORTACION','LINEA_NUEVA','ANTIGUEDAD','MALA_CALIDAD_GROSS','PROBA_CALIDAD_GROSS','CHURN_RATIO_DEALER','VIENE_POSPAGO'
,'TOTAL_QUEJAS','TOTAL_SEGUNDOS_COMPETENCIA', 'TOTAL_LLAMADAS_COMPETENCIA',
       'PROMEDIO_LLAMADA_COMPETENCIA','RECARGAS_FINDE',
       'RECARGAS_ENTRE_SEMANA','RECARGA_HORA_9_17',
       'RECARGA_HORA_18_8','CANTIDAD_RECARGAS',
       'VALOR_RECARGAS', 'PROMEDIO_RECARGA', 'VALOR_PRIMERA_RECARGA',
       'COMPRO_PAQUETE', 'CANTIDAD_PAQUETES', 'VALOR_PAQUETES',
       'PROMEDIO_PAQUETES','MB_TOTALES', 'DIAS_NAV','MENSAJES_COMPETENCIA'
]]

In [5]:
setup = setup(
    df_resultado,
    target = 'PORTACION',
    fix_imbalance=True
)

,Description,Value
0,session_id,1025
1,Target,PORTACION
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(1171334, 26)"
5,Missing Values,False
6,Numeric Features,21
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:

best_model = compare_models(sort='AUC')

IntProgress(value=0, description='Processing: ', max=74)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:42:53
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7655,0.7269,0.5900,0.0141,0.0276,0.0167,0.0629,186.9410
